In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

cpu


In [2]:
with open ("/Users/sebascordova/Desktop/Code proyects/wizard_of_oz.txt", 'r', encoding = 'utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
92


In [3]:

torch_1 = torch.rand(100, 100, 100, 100, device=device)
torch_2 = torch.rand(100, 100, 100, 100, device=device)


start_time = time.time()

result = torch.matmul(torch_1, torch_2)
end_time = time.time()
duration = end_time - start_time

print(f"Duración: {duration} segundos")

cpu
Duración: 0.48589515686035156 segundos


In [4]:
string_to_int = {ch:i for i, ch in enumerate (chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.sjoin([isnt_to_string[i]for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
data

tensor([91, 48, 65,  ...,  0,  0,  0])

In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    contexto = x[:t+1]
    tarjet = y[t]
    print('When input is', contexto, ' tarjet is ', tarjet)

When input is tensor([91])  tarjet is  tensor(48)
When input is tensor([91, 48])  tarjet is  tensor(65)
When input is tensor([91, 48, 65])  tarjet is  tensor(62)
When input is tensor([91, 48, 65, 62])  tarjet is  tensor(1)
When input is tensor([91, 48, 65, 62,  1])  tarjet is  tensor(44)
When input is tensor([91, 48, 65, 62,  1, 44])  tarjet is  tensor(75)
When input is tensor([91, 48, 65, 62,  1, 44, 75])  tarjet is  tensor(72)
When input is tensor([91, 48, 65, 62,  1, 44, 75, 72])  tarjet is  tensor(67)


# Basic PyTorch Functions

## Multinomial

In [22]:
probabilities = torch.tensor([0.1,0.9])
samples = torch.multinomial(probabilities, num_samples=10, replacement=True)
print(samples)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


## Cat

In [28]:
tensor = torch.tensor([1,2,3,4])
out =torch.cat((tensor,torch.tensor([5])), dim = 0)
out

tensor([1, 2, 3, 4, 5])

## tril - triu

In [32]:
out = torch.tril(torch.ones(5,5))
out

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [33]:
out = torch.triu(torch.ones(5,5))
out

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])

## masked_fill

In [39]:
out = torch.zeros(5,5).masked_fill(torch.tril(torch.ones(5,5)== 0), float('-inf'))
out

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

# transpose

In [41]:
input = torch.zeros(2,3,4)
out = input.transpose(0,2)
out.shape

torch.Size([4, 3, 2])

In [42]:
input = torch.zeros(2,3,4)
out = input.transpose(1,2)
out.shape

torch.Size([2, 4, 3])

In [49]:
sample = torch.tensor([10.,10.,10.])
linear = nn.Linear(3 , 3, bias=False)
print(linear(sample))

tensor([ 3.6430,  4.8599, -2.6513], grad_fn=<SqueezeBackward4>)


In [52]:
tensor1 = torch.tensor([1.0,2.0,3.0])
softmax_output = F.softmax(tensor1, dim=0)
print(softmax_output)

tensor([0.0900, 0.2447, 0.6652])


In [59]:
embedding_layer = nn.Embedding(num_embeddings=10000, embedding_dim=300)
input_indices =torch.LongTensor([1,5,3,2])

embedded_output = embedding_layer(input_indices)
print(embedded_output.shape)



torch.Size([4, 300])


In [62]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+batch_size +1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


tensor([ 70034, 128540,  60047, 188800])
inputs:
tensor([[76, 77, 62, 75, 76, 14,  3,  0],
        [58, 75, 61,  1, 58, 71, 61,  1],
        [ 1, 77, 65, 62,  0, 60, 58, 68],
        [68, 71, 66, 63, 62,  1, 63, 72]])
targets:
tensor([[77, 62, 75, 76],
        [75, 61,  1, 58],
        [77, 65, 62,  0],
        [71, 66, 63, 62]])
